(orc-chp-label)=

# ORC Combined Heat and Power

## Introduction

Subject of the following example of a more complex thermodynamic process will be a combined geothermal heat and power plant. As even deep geothermal boreholes can not supply temperatures high enough to drice conventional powerplant processes, such as the aforementioned and thoroughly analyzed Clausius Rankine Cycle, adjustments are necessary. Organic process fluids whose vaporization temperature is lower than that of water allow the execution of a Rankine cycle at these lower heat source temperatures, resulting in the so called *Organic Rankine Cycle*.

The following exercises are supposed to emulate a typical energy engineers workflow. The task is a process design study, where conditions of the surrundings and requirements of stakeholders are given. The proposed process design has to fulfill these constraints, which are displayed in {numref}`geothermal-chp-parameters`.

```{figure} /figures/geothermal_chp_parameters.svg
---
name: geothermal-chp-parameters
---
Boundary conditions of the geothermal plant with cogeneration of heat and power.
```

Additionally, {numref}`comp-approx` lists approximations of typical component parameters.

```{list-table} Approximations of typical component parameters
:header-rows: 1
:name: comp-approx

* - Component
  - Parameter
  - 
  - Value
  - Unit
* - Auxillary Installations
  - Electrical power requirements
  - 
  - 3
  - MW
* - Generators
  - Mechanical-electrical efficiency
  - 
  - 0.985
  - 
* - Motors
  - Electrical-mechanical efficiency
  - 
  - 0.975
  - 
* - Steam turbines
  - Isentropic efficiency
  - 
  - 0.9
  - 
* - 
  - Mechanical efficiency
  - 
  - 0.99
  - 
* - 
  - Minimum steam quality
  - 
  - 0.9
  - 
* - Pumps
  - Isentropic efficiency
  - 
  - 0.8
  - 
* - 
  - Mechanical efficiency
  - 
  - 0.99
  - 
* - Heat Exchangers
  - Minimal temperature difference
  - liquid-liquid
  - 10
  - K
* - 
  - 
  - liquid-condensing
  - 5
  - K
* - 
  - 
  - liquid-steam
  - 20
  - K
* - 
  - 
  - liquid-gaseous
  - 25
  - K
```

The proposed process is to be evaluated both with an energy- and an exergy-based approach. This can be achieved via efficiency values, which put the benefit of the process in relation to the necessary input. The energy efficiency of the cobined heat and power process is defined according to Eq. {eq}`orc-chp-energy-efficiency` and the exergy efficiency according to Eq. {eq}`orc-chp-exergy-efficiency`.

```{math}
    :label: orc-chp-energy-efficiency
    \eta_{CHP} = \frac{\dot{W}_{net} + \dot{Q}_{DH}}{\dot{H}_1 - \dot{H}_2}
```

```{math}
    :label: orc-chp-exergy-efficiency
    \epsilon_{CHP} = \frac{\dot{W}_{net} + (\dot{E}_{ff} - \dot{E}_{bf})}{\dot{E}_1 - \dot{E}_2}
```


## Exercise 1

1. Build a geothermal combined heat and power plant according to the boundary conditions in {numref}`geothermal-chp-parameters` and {numref}`comp-approx` and evaluate them according to Eqs. {eq}`orc-chp-energy-efficiency` and {eq}`orc-chp-exergy-efficiency`.
2. Build another geothermal plant based on the one before that replaces the heat output with another cycle extracting electrical power. Also evaluate this process and compare it with the combined heat and power plant.


```{Note}

This task is described very openly, so there is no single right solution. You have to be creative to find a working process that fulfills all requirements.

At first, you can try to come up with components that are necessary to supply the desired outputs. You can build simple cycles and contemplate how they can be improved.

Additionally, you can get inspiration through existing plants as described in the literature. A few examples are {cite:t}`DiPippo2015`, {cite:t}`Drbal1996`, {cite:t}`Kitto2005` and {cite:t}`Strauss2016` (german).

```

```{note}

Your model very likely will be quite complex and use a lot of components which need parametrization. Building the whole model at once likely results in too few or too many parameters, which can be hard to track down. Oftentimes, a better approach is to build parts of the model, e.g. inner cycles or outer strands, and make them work. Only then the parts are joined together to build the whole process.

Also, you will probably encouter issues when setting terminal temperature differences without stable starting values. As mentioned in other examples, running the model with best-guess starting values before finalizing parametrization strengthens stability and can even be necessary. Besides the other examples, you are referred to the TESPy tutorial on <a href="https://tespy.readthedocs.io/en/dev/tutorials/starting_values.html#tespy-tutorial-starting-values-label">How to Generate Stable Starting Values</a>.

```

````{admonition} Proposed process designs (only use if necessary!)
:class: dropdown

If you are having trouble creating/finding a suitable process design that fulfills all the requirements, example designs are provided here. Definitly try to come up with your own process designs at first, as there is a lot to learn trying, failing and hopefully finally succeeding in solving the tasks.

For the cogeneration of heat and power, an ORC using the liquid phase from the geothermal source will be employed to solely supply power. The gaseous mass fraction can be fed directly through a steam turbine and condensated to supply the district heating network with heat. The process diagram of the described CHP plant can be found in {numref}`orc-chp-flowsheet`. The heat outflow to close the Organic Rankine Cycle is realized through cooling with ambient air. Finally, the two streams from the geothermal source are combined again to be fed back into the geothermal sink.

```{figure} /figures/geothermal_orc_chp.svg
---
name: orc-chp-flowsheet
---
Flow sheet of the geothermal plant with cogeneration of heat and power.
```

The second approach without the supply of heat is implemented with an additional Organic Rankinke Cycle replacing the district heating condenser. This second ORC is in principle of the same structure as the first one, as can be seen in {numref}`orc-full-elec-flowsheet`.

```{figure} /figures/geothermal_orc_full_elec.svg
---
name: orc-full-elec-flowsheet
---
Flow sheet of the geothermal plant with sole power generation.
```

````

## Proposed solution 1

**First Step**: Build up the ORC cycle as basis for geothermal combined heat and power plant and geothermal power plant

In [207]:
from tespy.networks import Network
from tespy.connections import Connection, Bus
from tespy.components import (
    CycleCloser, Source, Sink, Pump, Turbine, HeatExchanger,HeatExchangerSimple,
    Condenser, Drum, DropletSeparator, Merge
    )

# Network
nw = Network(fluids=['water', 'air', 'Isopentane'])
nw.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

flu_vec_orc = {'water': 0, 'air': 0, 'Isopentane': 1}
flu_vec_water = {'water': 1, 'air': 0, 'Isopentane': 0}
flu_vec_air = {'water': 0, 'air': 1, 'Isopentane': 0}

# Components
# ORC
orc_cc = CycleCloser('ORC Cycle Closer')
orc_preheater = HeatExchanger('ORC Preheater')
orc_drum = Drum('ORC Drum')
orc_evaporator = HeatExchanger('ORC Evaporator')
orc_turbine = Turbine('ORC Turbine')
orc_condenser = Condenser('ORC Condenser')
orc_pump = Pump('ORC Pump')

# Condenser coolant cycle
cool_source = Source('Coolant Source')
cool_sink = Sink('Coolant Sink')

# Auxiliary source and sink
liquid_source = Source('Liquid Sep Source')
geo_pump = Pump('Geothermal Pump')
geo_sink1 = Sink('Geothermal Sink1')

# Connections
# ORC
orc_pump2orc_cc = Connection(orc_pump, 'out1', orc_cc, 'in1', label='0')
orc_cc2orc_preheater = Connection(orc_cc, 'out1', orc_preheater, 'in2', label='1')
orc_preheat2orc_drum = Connection(orc_preheater, 'out2', orc_drum, 'in1', label='2')
orc_drum2orc_eva = Connection(orc_drum, 'out1', orc_evaporator, 'in2', label='3')
orc_eva2orc_drum = Connection(orc_evaporator, 'out2', orc_drum, 'in2', label='4')
orc_drum2orc_turb = Connection(orc_drum, 'out2', orc_turbine, 'in1', label='5')
orc_turb2orc_cond = Connection(orc_turbine, 'out1', orc_condenser, 'in1', label='6')
orc_cond2orc_pump = Connection(orc_condenser, 'out1', orc_pump, 'in1', label='7')

nw.add_conns(
    orc_pump2orc_cc, orc_cc2orc_preheater, orc_preheat2orc_drum, orc_drum2orc_eva,
    orc_eva2orc_drum, orc_drum2orc_turb, orc_turb2orc_cond, orc_cond2orc_pump
    )

# Condenser coolant cycle
cool_source2orc_cond = Connection(cool_source, 'out1', orc_condenser, 'in2', label='41')
orc_cond2cool_sink = Connection(orc_condenser, 'out2', cool_sink, 'in1', label='42')

nw.add_conns(cool_source2orc_cond, orc_cond2cool_sink)

# Geothermal liquid cyclye
liquid_source2orc_eva = Connection(liquid_source, 'out1', orc_evaporator, 'in1', label='22')
orc_eva2orc_preheat = Connection(orc_evaporator, 'out1', orc_preheater, 'in1', label='23')
orc_preheat2geo_pump = Connection(orc_preheater, 'out1', geo_pump, 'in1', label='24')
geo_pump2geo_sink1 = Connection(geo_pump, 'out1', geo_sink1, 'in1', label='26')

nw.add_conns(
    liquid_source2orc_eva, orc_eva2orc_preheat, orc_preheat2geo_pump, geo_pump2geo_sink1
    )

**Second Step**:  Set the process parameters and starting values. After that solve orc cycle with starting and final values

In [208]:
from CoolProp.CoolProp import PropsSI as PSI

# Parametrization
# Component parameters
orc_turbine.set_attr(eta_s=0.9)

orc_pump.set_attr(eta_s=0.8)
geo_pump.set_attr(eta_s=0.8)

orc_preheater.set_attr(pr1=1, pr2=1)
orc_evaporator.set_attr(pr1=1)
orc_condenser.set_attr(pr1=1, pr2=1)

# Connection parameter
# ORC
p_orc_condenser = PSI("P", "Q", 0, "T", 35 + 273.15, 'Isopentane') * 1e-5
orc_turb2orc_cond.set_attr(p=p_orc_condenser, fluid=flu_vec_orc)

p_orc_evaporator = PSI("P", "Q", 1, "T", 150 + 273.15, 'Isopentane') * 1e-5
orc_eva2orc_drum.set_attr(p=p_orc_evaporator, x=0.7)

orc_preheat2orc_drum.set_attr(Td_bp=-5)

# Geothermal liquid cyclye
liquid_source2orc_eva.set_attr(m=500*0.55, p=10.8, x=0, fluid=flu_vec_water)
geo_pump2geo_sink1.set_attr(T=55, p=30)

# Condenser coolant cycle
cool_source2orc_cond.set_attr(T=15, p=1.013, fluid=flu_vec_air)
orc_cond2cool_sink.set_attr(T=30)

# Initial solve with starting values
nw.solve("design")

# Final solve
orc_eva2orc_drum.set_attr(p=None)
orc_evaporator.set_attr(ttd_l=10)

orc_turb2orc_cond.set_attr(p=None)
orc_condenser.set_attr(ttd_u=5)

nw.solve("design")

iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 3.06e+08 | 3.28e+05 | 7.93e+06 | 5.66e+06 | 0.00e+00
2	| 6.19e+09 | 2.79e+06 | 3.09e+06 | 1.29e+08 | 0.00e+00
3	| 3.64e+10 | 1.93e+06 | 1.27e+06 | 2.80e+07 | 0.00e+00
4	| 1.76e+10 | 4.08e+05 | 5.57e+05 | 1.06e+06 | 0.00e+00
5	| 3.14e+09 | 1.47e+05 | 2.13e+05 | 5.90e+06 | 0.00e+00
6	| 1.43e+08 | 3.17e+03 | 5.50e+04 | 1.41e+05 | 0.00e+00
7	| 2.81e+05 | 3.78e+00 | 4.04e-10 | 3.00e+02 | 0.00e+00
8	| 5.85e-01 | 7.11e-05 | 4.03e-10 | 6.03e-04 | 0.00e+00
9	| 8.21e-08 | 6.36e-10 | 4.04e-10 | 1.18e-08 | 0.00e+00


--------+----------+----------+----------+----------+---------
Total iterations: 9, Calculation time: 0.2 s, Iterations per second: 49.57
iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 3.00e+01 | 7.49e+02 | 6.23e+06 | 4.63e+05 | 0.00e+00
2	| 1.46e+07 | 3.50e+02 | 1.93e+06 | 5.16e+04 | 0.00e+00
3	| 2.39e+06 | 2.79e+02 | 3.88e+05 | 1.47e+04 | 0.00e+00
4	| 1.42e+05 | 6.76e+01 | 2.06e+04 | 6.34e+03 | 0.00e+00
5	| 6.85e+03 | 1.06e+00 | 9.05e+01 | 8.46e+01 | 0.00e+00
6	| 1.16e+00 | 1.27e-04 | 4.49e-02 | 1.43e-02 | 0.00e+00
7	| 7.22e-07 | 1.07e-05 | 7.90e-03 | 2.06e-03 | 0.00e+00
--------+----------+----------+----------+----------+---------
Total iterations: 7, Calculation time: 0.3 s, Iterations per second: 24.93


## Proposed solution 1.1

### Geothermal combined heat and power plant

**First Step**: Build up the steam cycle for the chp process design

Be sure this one runs standalone first!

In [209]:
# Steam cycle with district heating system
# Components
steam_turbine = Turbine('Steam Turbine')
dh_condenser = Condenser('District Heating Condenser')
dh_preheater = HeatExchanger('District Heating Preheater')
dh_cc = CycleCloser('District Heating Cycle Closer')
dh_pump = Pump('District Heating Pump')
dh_heatsink = HeatExchangerSimple('District Heating Consumer')
steam_pump = Pump('Steam Pump')

# Auxiliary source and sink
steam_source = Source('Steam Sep Source')
geo_sink2 = Sink('Geothermal Sink2')

# Connection
# Steam system
steam_source2steam_turb = Connection(steam_source, 'out1', steam_turbine, 'in1', label='11')
steam_turb2dh_cond = Connection(steam_turbine, 'out1', dh_condenser, 'in1', label='12')
dh_cond2dh_preheat = Connection(dh_condenser, 'out1', dh_preheater, 'in1', label='13')
dh_preheat2dh_cond = Connection(dh_preheater, 'out1', steam_pump, 'in1', label='14')
steam_pump2geo_sink2 = Connection(steam_pump, 'out1', geo_sink2, 'in1', label='15')

nw.add_conns(
    steam_source2steam_turb, steam_turb2dh_cond, dh_cond2dh_preheat,
    dh_preheat2dh_cond, steam_pump2geo_sink2
    )

# District heating system
dh_heatsink2dh_cc = Connection(dh_heatsink, 'out1', dh_cc, 'in1', label='30')
dh_cc2dh_preheat = Connection(dh_cc, 'out1', dh_preheater, 'in2', label='31')
dh_preheat2dh_cond = Connection(dh_preheater, 'out2', dh_condenser, 'in2', label='33')
dh_cond2dh_heatsink = Connection(dh_condenser, 'out2', dh_heatsink, 'in1', label='34')

nw.add_conns(
    dh_heatsink2dh_cc, dh_cc2dh_preheat, dh_preheat2dh_cond,
    dh_cond2dh_heatsink
    )

# Parametrization
# Components parameter
steam_turbine.set_attr(eta_s=0.9)
steam_pump.set_attr(eta_s=0.8)

dh_condenser.set_attr(pr1=1, pr2=1)
dh_preheater.set_attr(pr1=1, pr2=1)

# Connection parameter
# Steam system
p_dh_condenser = PSI("P", "Q", 0, "T", 105 + 273.15, 'water') * 1e-5
steam_turb2dh_cond.set_attr(p=p_dh_condenser)

steam_source2steam_turb.set_attr(m=500*0.45, p=10.8, x=1, fluid=flu_vec_water)
steam_pump2geo_sink2.set_attr(T=55, p=30)

# District heating system
dh_cond2dh_heatsink.set_attr(T=100, p=10, fluid=flu_vec_water)
dh_heatsink2dh_cc.set_attr(T=50)

nw.solve('design')

iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 8.86e+06 | 1.81e+03 | 7.91e+06 | 1.88e+08 | 0.00e+00
2	| 7.61e+09 | 6.50e+03 | 3.39e+06 | 1.72e+07 | 0.00e+00
3	| 2.13e+10 | 3.40e-03 | 1.29e+06 | 6.34e+06 | 0.00e+00
4	| 5.93e+05 | 6.41e-04 | 5.42e+05 | 8.41e+04 | 0.00e+00
5	| 2.47e+05 | 2.83e-05 | 1.95e+05 | 6.22e+04 | 0.00e+00
6	| 5.02e+04 | 1.77e-05 | 3.53e+04 | 3.41e+04 | 0.00e+00
7	| 3.09e+03 | 4.61e-09 | 3.37e-06 | 3.11e+03 | 0.00e+00
8	| 1.79e-07 | 2.16e-08 | 1.59e-05 | 4.58e-06 | 0.00e+00
--------+----------+----------+----------+----------+---------
Total iterations: 8, Calculation time: 0.2 s, Iterations per second: 36.0


**Second Step**: Connecting ORC with Steam Cycle

1. Build up the merge part
2. Build up the seperator part
3. Set busses und solve the geothermal combined heat and power plant
4. Create and execute `ExergyAnalysis` instance

In [210]:
from tespy.tools import ExergyAnalysis

# Components
# Geothermal heat source with interfaces
geo_source = Source('Geothermal Source')
geo_sink = Sink('Geothermal Sink')
seperator = DropletSeparator('Seperator')
merge = Merge('Merge')

# Build up the merge part
# Delete old connections
nw.del_conns(geo_pump2geo_sink1, steam_pump2geo_sink2, orc_preheat2geo_pump)

# Set new connections
steam_pump2merge = Connection(steam_pump, 'out1', merge, 'in1', label='15')
orc_preheat2merge = Connection(orc_preheater, 'out1', merge, 'in2', label='24')
merge2geo_pump = Connection(merge, 'out1', geo_pump, 'in1', label='25')
geo_pump2geo_sink = Connection(geo_pump, 'out1', geo_sink, 'in1', label='26')

nw.add_conns(steam_pump2merge, orc_preheat2merge, merge2geo_pump, geo_pump2geo_sink)

# Parametrize new connections
orc_preheat2merge.set_attr(T=55)
geo_pump2geo_sink.set_attr(p=30)
dh_preheater.set_attr(ttd_l=5)

# Build up the seperator part
# Delete old connections
nw.del_conns(liquid_source2orc_eva, steam_source2steam_turb)

# Set new connections
geo_source2sep = Connection(geo_source,'out1', seperator, 'in1', label='21')
sep2orc_eva = Connection(seperator,'out1', orc_evaporator, 'in1', label='22')
sep2steam_turb = Connection(seperator, 'out2', steam_turbine, 'in1', label='11')

nw.add_conns(geo_source2sep, sep2orc_eva, sep2steam_turb)

# Parametrize new connections
geo_source2sep.set_attr(m=500, p=10.8, x=0.45, fluid=flu_vec_water)
sep2orc_eva.set_attr(m0=500*0.55)
sep2steam_turb.set_attr(m0=500*0.45)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps(
    {'comp': geo_source, 'base': 'bus'},
    {'comp': geo_sink, 'base': 'component'}
    )

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': steam_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': orc_turbine, 'char': 0.985*0.99, 'base': 'component'},
    {'comp': steam_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': orc_pump, 'char': 0.975*0.99, 'base': 'bus'},
    {'comp': geo_pump, 'char': 0.975*0.99, 'base': 'bus'}
    )

heat_out_dh = Bus('Heat Output District Heating')
heat_out_dh.add_comps({'comp': dh_heatsink, 'base': 'component'})

heat_out_cool = Bus('Heat Output Cooling')
heat_out_cool.add_comps(
    {'comp': cool_source, 'base': 'bus'},
    {'comp': cool_sink, 'base': 'component'}
    )

nw.add_busses(heat_in, power_out, heat_out_dh, heat_out_cool)

# Solve model
nw.solve("design")
nw.print_results()

# Calculate thermal efficiency
h_in = PSI('H', 'Q', 0.45, 'P', 10.8e5, 'water')
h_out = PSI('H', 'T', 55 + 273.15, 'P', 30e5, 'water')
delta_H = 500 * (h_out - h_in)
eta_chp = (abs(power_out.P.val)+heat_out_dh.P.val)/delta_H
print(f'eta_chp = {eta_chp:.3f}')

# Create and execute exergy analysis
dh_heatsink.set_attr(dissipative=False)
ean = ExergyAnalysis(
    network=nw, E_F=[heat_in], E_P=[heat_out_dh, power_out], E_L=[heat_out_cool])
ean.analyse(pamb=1.013, Tamb=15)
# ean.print_results(components=False, connections=False, busses=False, groups=False)
# ean.print_results()
epsilon_chp = ean.network_data['epsilon']
print(f'epsilon_chp = {epsilon_chp:.3f}')

iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 6.48e+08 | 1.56e+03 | 2.87e+06 | 1.45e+07 | 5.34e-14
2	| 1.25e+08 | 1.62e+03 | 4.78e+05 | 3.35e+05 | 1.61e-16
3	| 1.35e+06 | 3.08e+02 | 7.41e+03 | 8.53e+03 | 7.67e-19
4	| 1.95e+04 | 5.73e-01 | 8.80e+02 | 2.48e+02 | 1.61e-16
5	| 7.64e+00 | 5.13e-04 | 2.01e-01 | 4.93e-02 | 4.86e-34
6	| 1.82e-06 | 1.95e-04 | 1.48e-01 | 3.80e-02 | 2.02e-34
--------+----------+----------+----------+----------+---------
Total iterations: 6, Calculation time: 0.2 s, Iterations per second: 24.55
##### RESULTS (Turbine) #####
+---------------+-----------+----------+----------+
|               |         P |    eta_s |       pr |
|---------------+-----------+----------+----------|
| ORC Turbine   | -1.13e+07 | 9.00e-01 | 4.00e-01 |
| Steam Turbine | -7.60e+07 | 9.00e-01 | 1.12e-01 |
+---------------+-----------+----------+----------+
##### RESULTS (Condenser) #####
+-------------------------

## Proposed solution 1.2

### geothermal plant with only power generation

**First Step**: Build up the low pressure ORC for complete condensation operation

Be sure this one runs standalone first!

In [211]:
nw_el = Network(fluids=['water', 'air', 'Isopentane'])
nw_el.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

flu_vec_orc = {'water': 0, 'air': 0, 'Isopentane': 1}
flu_vec_water = {'water': 1, 'air': 0, 'Isopentane': 0}
flu_vec_air = {'water': 0, 'air': 1, 'Isopentane': 0}

# Components
# Low pressure ORC
lp_orc_cc = CycleCloser('LP ORC Cycle Closer')
lp_orc_preheater = HeatExchanger('LP ORC Preheater')
lp_orc_drum = Drum('LP ORC Drum')
lp_orc_evaporator = HeatExchanger('LP ORC Evaporator')
lp_orc_turbine = Turbine('LP ORC Turbine')
lp_orc_condenser = Condenser('LP ORC Condenser')
lp_orc_pump = Pump('LP ORC Pump')

# Condenser coolant cycle
lp_cool_source = Source('LP Coolant Source')
lp_cool_sink = Sink('LP Coolant Sink')

# Steam Cycle
steam_turbine = Turbine('Steam Turbine')
steam_pump = Pump('Steam Pump')

# Auxiliary source and sink
steam_source = Source('Steam Sep Source')
liquid_sink = Sink('Liquid Sink')

# Connections
# Low pressure ORC
lp_orc_pump2lp_orc_cc = Connection(lp_orc_pump, 'out1', lp_orc_cc, 'in1', label='40')
lp_orc_cc2lp_orc_preheater = Connection(lp_orc_cc, 'out1', lp_orc_preheater, 'in2', label='41')
lp_orc_preheat2lp_orc_drum = Connection(lp_orc_preheater, 'out2', lp_orc_drum, 'in1', label='42')
lp_orc_drum2lp_orc_eva = Connection(lp_orc_drum, 'out1', lp_orc_evaporator, 'in2', label='43')
lp_orc_eva2lp_orc_drum = Connection(lp_orc_evaporator, 'out2', lp_orc_drum, 'in2', label='44')
lp_orc_drum2lp_orc_turb = Connection(lp_orc_drum, 'out2', lp_orc_turbine, 'in1', label='45')
lp_orc_turb2lp_orc_cond = Connection(lp_orc_turbine, 'out1', lp_orc_condenser, 'in1', label='46')
lp_orc_cond2lp_orc_pump = Connection(lp_orc_condenser, 'out1', lp_orc_pump, 'in1', label='47')

nw_el.add_conns(
    lp_orc_pump2lp_orc_cc, lp_orc_cc2lp_orc_preheater, lp_orc_preheat2lp_orc_drum, 
    lp_orc_drum2lp_orc_eva, lp_orc_eva2lp_orc_drum, lp_orc_drum2lp_orc_turb, 
    lp_orc_turb2lp_orc_cond, lp_orc_cond2lp_orc_pump
    )

# Condenser coolant cycle
lp_cool_source2lp_orc_cond = Connection(lp_cool_source, 'out1', lp_orc_condenser, 'in2', label='61')
lp_orc_cond2lp_cool_sink = Connection(lp_orc_condenser, 'out2', lp_cool_sink, 'in1', label='62')

nw_el.add_conns(lp_cool_source2lp_orc_cond, lp_orc_cond2lp_cool_sink)

# steam cyclye
steam_source2steam_turb = Connection(steam_source, 'out1', steam_turbine, 'in1', label='12')
steam_turb2lp_orc_eva = Connection(steam_turbine, 'out1', lp_orc_evaporator, 'in1', label='13')
lp_orc_eva2lp_orc_preheat = Connection(lp_orc_evaporator, 'out1', lp_orc_preheater, 'in1', label='14')
lp_orc_preheat2steam_pump = Connection(lp_orc_preheater, 'out1', steam_pump, 'in1', label='15')
steam_pump2liquid_sink = Connection(steam_pump, 'out1', liquid_sink, 'in1', label='16')

nw_el.add_conns(
    steam_source2steam_turb, steam_turb2lp_orc_eva, lp_orc_eva2lp_orc_preheat,
    lp_orc_preheat2steam_pump, steam_pump2liquid_sink
    )

**Second Step**: Set the process parameters and starting values. After that solve orc cycle with starting and final values

In [212]:
from CoolProp.CoolProp import PropsSI as PSI

# Parametrization
# Component parameters
steam_pump.set_attr(eta_s=0.9)
lp_orc_turbine.set_attr(eta_s=0.9)

lp_orc_pump.set_attr(eta_s=0.8)
steam_pump.set_attr(eta_s=0.8)

lp_orc_preheater.set_attr(pr1=1, pr2=1)
lp_orc_evaporator.set_attr(pr1=1)
lp_orc_condenser.set_attr(pr1=1, pr2=1)

# Connection parameter
# Low pressure ORC
p_lp_orc_condenser = PSI("P", "Q", 0, "T",  35 + 273.15, 'water') * 1e-5
lp_orc_turb2lp_orc_cond.set_attr(p=p_lp_orc_condenser, fluid=flu_vec_orc)

p_lp_orc_evaporator = PSI("P", "Q", 1, "T", 85 + 273.15, 'Isopentane') * 1e-5
lp_orc_eva2lp_orc_drum.set_attr(p=p_lp_orc_evaporator, x=0.7)

lp_orc_preheat2lp_orc_drum.set_attr(Td_bp=-5)

# Geothermal liquid cyclye
steam_source2steam_turb.set_attr(m=500*0.45, p=10.8, x=1, fluid=flu_vec_water)
# lp_orc_preheat2steam_pump.set_attr(T=30, p=2)
steam_pump2liquid_sink.set_attr(T=55, p=30)

# Condenser coolant cycle
lp_cool_source2lp_orc_cond.set_attr(T=15, p=1.013, fluid=flu_vec_air)
lp_orc_cond2lp_cool_sink.set_attr(T=30)

# Initial solve with starting values
nw_el.solve("design")
nw_el.print_results()

# # Final solve
# orc_eva2orc_drum.set_attr(p=None)
# orc_evaporator.set_attr(ttd_l=10)

# orc_turb2orc_cond.set_attr(p=None)
# orc_condenser.set_attr(ttd_u=5)

# nw_el.solve("design")


ERROR:root:Singularity in jacobian matrix, calculation aborted! Make sure your network does not have any linear dependencies in the parametrisation. Other reasons might be
-> given temperature with given pressure in two phase region, try setting enthalpy instead or provide accurate starting value for pressure.
-> given logarithmic temperature differences or kA-values for heat exchangers, 
-> support better starting values.
-> bad starting value for fuel mass flow of combustion chamber, provide small (near to zero, but not zero) starting value.


iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 1.02e+07 |      nan |      nan |      nan |      nan
--------+----------+----------+----------+----------+---------
Total iterations: 1, Calculation time: 0.0 s, Iterations per second: 31.16
##### RESULTS (Connection) #####
+----+-------------+-------------+-----+-------------+
|    |           m |           p |   h |           T |
|----+-------------+-------------+-----+-------------|
| 40 | nan         | nan         | nan | nan         |
| 41 | nan         | nan         | nan | nan         |
| 42 | nan         | nan         | nan | nan         |
| 43 | nan         | nan         | nan | nan         |
| 44 | nan         |   5.154e+00 | nan | nan         |
| 45 | nan         | nan         | nan | nan         |
| 46 | nan         |   5.629e-02 | nan | nan         |
| 47 | nan         | nan         | nan | nan         |
| 61 | nan         |   1.013e+00 | nan |   1.50

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Exercise 2

- Energetische

- Treffe eine Auswahl von geeigneten Arbeitsmedien 
- Auswahl und Begründung der Nutzung thermodynamischer Stoffdatenmodelle für die Simulation.

In [ ]:
# code of solution

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Exercise 3

- Definition und Nutzung geeigneter Größen zur Bewertung der Prozessentwürfe
- anschließender Vergleich sowie Diskussion.

In [ ]:
# code of solution

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Lessons Learned

- Lessons Learned 1
- Lessons Learned 2
- Lessons Learned 3